In [1]:
%load_ext autoreload
%autoreload 2

# Tutorial on the basic usage of `esm4ppe`
The `esm4ppe` package is meant to offer easy control and access to a set of perfect model experiments performed with ESM4.1. The package has three primary uses/purposes:
1. Basic functionality to load ensemble and control data (including retrieving from tape).
2. Configure loaded data to match the required format of the [`climpred` package](https://climpred.readthedocs.io/en/stable/) such that all of the associated functionality of that package can be applied to the ESM4 data.
3. Implement checks and procedures to ensure that calculations/data loading are not repeated unnecessarily.

In [2]:
import esm4ppe

## The `esm4ppe` object
The highest level functionality of the package is mediated through the `esm4ppeObj` class. This object takes as arguments just the variable name and the temporal frequency of interest. In the background, it finds all of the relevant paths to the ensemble and control data, as well as the grid data (static files).

In [3]:
variable,frequency = 'intpp','monthly'
es = esm4ppe.esm4ppeObj(variable,frequency)

Opening static... static opened.


/home/graemem/.conda/envs/climpred/lib/python3.11/site-packages/xarray/conventions.py:448: SerializationWarning: variable 'thkcello' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/graemem/.conda/envs/climpred/lib/python3.11/site-packages/xarray/conventions.py:448: SerializationWarning: variable 'volcello' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


To this object, we can add the data for the ensemble using the `.add_ensemble()` function. In adding the ensemble, the package will perform a few preliminary checks:
1. Has the ensemble for this variable and this frequency already been opened, preprocessed and saved to a zarr store (the location of which is specified by the user in `version.py`; see the package README for details). If so, it loads the data from this location.
2. If the zarr store does not exist, has the user has specified `triggeropen = True` in the arguments issued to `.add_ensemble()`. Since loading the ensemble from the original netcdf files takes a decently long time, this prevents the user from accidentally trying to open files that they don't wish to.
3. If the user has specified `triggeropen = True`, it tries to open the ensemble, but first checks if the files have been retrieved from tape. If not, it recommends that the user issue a `dmget` command for the relevant files. This can be done using the `esm4ppeObj` object (see below).
4. Finally, it opens the ensemble and saves it to a new zarr store.

The basic syntax is

In [4]:
es = es.add_ensemble(triggeropen=True)

Ensemble present in zarr store... opening... ensemble opened.


In much the same way, we can add the control simulation:

In [6]:
es = es.add_control(triggeropen=True)

Control present in zarr store... opening... control opened.


The ensemble and control are now available within the `esm4ppeObj` as `xarray` datasets, called `.ensemble` and `.control` respectively. Note that the dimensional information of this dataset is consistent with the format of the `climpred` package, including `init` to specify the initialization time of each ensemble, `member` to specify the member number, and `lead` to specify the lead time following the time of initialization (in units of the temporal frequency).

In [5]:
es.ensemble

<xarray.Dataset>
Dimensions:  (init: 40, lead: 120, member: 10, xh: 720, yh: 576)
Coordinates:
  * init     (init) object 0123-01-01 00:00:00 ... 0381-10-01 00:00:00
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 ... 113 114 115 116 117 118 119 120
  * member   (member) int64 0 1 2 3 4 5 6 7 8 9
  * xh       (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh       (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Data variables:
    intpp    (member, init, lead, yh, xh) float32 dask.array<chunksize=(10, 40, 1, 576, 145), meta=np.ndarray>

### Issuing `dmget`
For a given variable and frequency, retrieving the relevant files from tape can be done using the `.issue_dmget()` function included as part of the `esm4ppeObj`. Without arguments, `dmget` will be issued for both the ensemble and control, or either of these can be specified with the `dataset` keyword (e.g. `dataset=ensemble`). Additionally, the user can specify `wait=True` such that the command will not complete until the `dmget` process has completed. This can be useful if you want to specify subsequent commands (e.g. adding the ensemble) but this depends on these files being retrieved from tape. Note that the waiting is done in a rather rudimentary way, simply looking for the user's name in the `dmget` queue, such that it's possible that it would get confused if the user was retrieving other data at the same time.

### Verification of skill
Tests of predictability can be performed using the `.verify()` function in the `esm4ppeObj`. This uses the exact functionality as the equivalent function in the `climpred` package (thus, see this package for all options and metrics), with two additional components:
1. It checks whether the calculation has been done already, and if so loads from that saved dataset (with the location again set in `version.py`).
2. It includes additional metrics, such as the potential prognostic predictability (ppp), that are not presently included in `climpred`.

So, for example, to calculate the ppp:

In [8]:
es = es.verify('ppp',saveskill=True,groupby='month')

/home/graemem/.conda/envs/climpred/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 10 times more chunks
  return self.array[key]


[#####                                   ] | 12% Completed | 78.43 ss

/home/graemem/.conda/envs/climpred/lib/python3.11/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 10m 6ss
...skill metric saved


Note that the `groupby` keyword comes from the `climpred` package and separates initialization dates based on their month.

The skill dataset is both saved to the relevant directory and stored in the `esm4ppeObj` as `.vs`

In [9]:
es.vs

<xarray.Dataset>
Dimensions:  (lead: 120, xh: 720, yh: 576, month: 4)
Coordinates:
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 ... 113 114 115 116 117 118 119 120
  * xh       (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh       (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * month    (month) int64 1 4 7 10
Data variables:
    intpp    (month, lead, yh, xh) float32 dask.array<chunksize=(1, 1, 576, 145), meta=np.ndarray>

### Other operations
Functions to perform additional operations on the ensemble, control, or skill datasets --- for example to calculate regional means or climatologies --- are under development. What is currently available has been developed for use with ocean variables and is likely to be buggy for other model components.

## Direct use of functions
Separate from the high level functionality of the `esm4ppeObj`, you also have access to lower level functions as part of the package. Here are some examples:

In [9]:
# Get the postprocess path for a specific ensemble member for a specific initialization year:
path = esm4ppe.get_pp(startyear=161,member=2)
path

'/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/ESM4_piControl_D-ensemble-0161*01-02/gfdl.ncrc4-intel18-prod-openmp/pp'

In [10]:
# To expand the wildcard
import glob
glob.glob(path)

['/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/ESM4_piControl_D-ensemble-01610101-02/gfdl.ncrc4-intel18-prod-openmp/pp',
 '/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/ESM4_piControl_D-ensemble-01610401-02/gfdl.ncrc4-intel18-prod-openmp/pp',
 '/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/ESM4_piControl_D-ensemble-01610701-02/gfdl.ncrc4-intel18-prod-openmp/pp',
 '/archive/Richard.Slater/xanadu_esm4_20190304_mom6_ESM4_v1.0.3_rc1/ESM4_piControl_D-ensemble-01611001-02/gfdl.ncrc4-intel18-prod-openmp/pp']

In [11]:
# Get the postprocess file name for a given variable and frequency:
esm4ppe.get_ppname('evap_land','daily')

'land_daily'

In [12]:
# Open a specific ensemble dataset from the netcdf file, given the variable, frequency, and initialization information:
ensemble = esm4ppe.open_ensemble('intpp','monthly',startyear=161,startmonth=4,controlasmember=False)
ensemble

<xarray.Dataset>
Dimensions:    (member: 9, init: 1, xh: 720, yh: 576, lead: 36, nv: 2)
Coordinates:
  * member     (member) int64 1 2 3 4 5 6 7 8 9
  * init       (init) object 0161-04-01 00:00:00
  * xh         (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh         (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * lead       (lead) int64 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
  * nv         (nv) float64 1.0 2.0
Data variables:
    intpp      (member, init, lead, yh, xh) float32 dask.array<chunksize=(1, 1, 36, 576, 720), meta=np.ndarray>
    time_bnds  (member, init, lead, nv) object dask.array<chunksize=(1, 1, 36, 2), meta=np.ndarray>
Attributes:
    filename:          01610401.ocean_cobalt_omip_2d.nc
    title:             ESM4_piControl_D-ensemble-01610401-01
    associated_files:  areacello: 01610401.ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A
    history:           2022-06-11 18:56:49 -0400: PostProcessing::PPNetCDF::t...

In [13]:
# Open the control simulation for a given variable and frequency:
control = esm4ppe.open_control('intpp','monthly')
control

<xarray.Dataset>
Dimensions:    (xh: 720, yh: 576, time: 3600, nv: 2)
Coordinates:
  * xh         (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh         (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * time       (time) object 0101-01-16 12:00:00 ... 0400-12-16 12:00:00
  * nv         (nv) float64 1.0 2.0
Data variables:
    intpp      (time, yh, xh) float32 dask.array<chunksize=(60, 576, 720), meta=np.ndarray>
    time_bnds  (time, nv) object dask.array<chunksize=(60, 2), meta=np.ndarray>
Attributes:
    filename:          01010101.ocean_cobalt_omip_2d.nc
    title:             ESM4_piControl_D
    associated_files:  areacello: 01010101.ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A
    history:           2022-03-14 16:55:28 -0400: PostProcessing::PPNetCDF::t...

## Python scripts to process a set of variables
It is likely the case that we have some specific variables of interest and we would like to created zarr stores for the ensemble and control for each of these. With this set-up, this only needs to be done once, and can be easily written in a python script and submitted as a batch job. An example script is provided: `example_preprocess.py`.